<br><br>

<font color=#191347 ><font>
<p style = "font-family:times;">
    <font size="7">
        Interactive Learning 
    </font>    
</p>

<br>
    
<font color=#ed1c24><font>
<p style = "font-family:optima;">
    <font size="6">
        Homework #3
    </font>    
<font color=#000000> <font>
</p>

> $\textit{Mohammad Saadati - 810198410}$

# Table of Contents

- [Packages](#0)
- [Problem 2](#1)
    - [Question 1](#1-1)
    - [Question 2](#1-2)
    - [Question 3](#1-3)
    - [Question 4](#1-4)
- [Problem 3](#2)
    - [َPart 1](#2-1)
        - [Value Iteration](#2-1-1)
        - [Policy Iteration](#2-1-2)
    - [Part 2](#2-2)
        - [Value Iteration](#2-2-1)
        - [Policy Iteration](#2-2-2)
    - [Part 3](#2-3)
        - [A](#2-3-1)
        - [B](#2-3-2)

<a name='0'></a>
## Packages

Run the following cell to load the packages you'll need.

In [1]:
import numpy as np
import itertools
import copy
import time
from gym import Env

from env import FrozenLake

# Action Space 
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
X = 0
Y = 1

ACTION_SPACE = [LEFT,DOWN,RIGHT,UP]
ACTIONS_NAMES = ["←","↓","→","↑"]

Discount = 0.9
Sd_number = 810198410

<a name='1'></a>
# Problem 2

<a name='1-1'></a>
## Question 1

<a name='1-2'></a>
## Question 2

<a name='1-3'></a>
## Question 3

<a name='1-4'></a>
## Question 4

<a name='2'></a>
# Problem 3

In [2]:
def show_states_value_on_map(states_value, map_size):
    environment_map = ""
    for i in range(map_size):
        environment_map += ("\n"+ int(10*map_size)*"-" + "\n| ")
        for j in range(map_size):
            environment_map += "{:.4f} | ".format(states_value[i,j])           
    environment_map += ("\n" + int(10*map_size)*"-")
    return environment_map

def show_actions_on_map(iteration, map_size):
    environment_map = ""
    for i in range(map_size):
        environment_map += ("\n"+ int(4.3*map_size)*"-" + "\n| ")
        for j in range(map_size):
            if (i,j)!=(map_size-1,map_size-1):
                environment_map += "{D} | ".format(D=ACTIONS_NAMES[iteration.get_optimal_policy([i, j])])
            else:
                environment_map += "* | "          
    environment_map += ("\n" + int(4.3*map_size)*"-")
    return environment_map

In [36]:
class ValueIteration():
    def __init__(self, env, discount, theta, map_size):
        self.env = env
        self.discount = discount
        self.theta = theta
        self.map_size = map_size
        self.delta = 0
        self.state_values = np.zeros((self.map_size,self.map_size))
        self.q_values = np.zeros((self.map_size,self.map_size,4))
    
    def value_estimation(self):
        self.delta = np.inf
        
        while(self.delta > self.theta):
            
            self.delta = 0
            
            for state in itertools.product(range(self.map_size), range(self.map_size)):
                if (state[X],state[Y]) == (self.map_size-1,self.map_size-1):
                    continue
                    
                for action in ACTION_SPACE:
                    next_states, states_probability, breaking_possibility, is_end = self.env.find_next_states(action=action, current_state=(state[X],state[Y]))
                    reward = np.where(is_end, 99, ((1-breaking_possibility) * -1) + (breaking_possibility * -11))
                    next_states = np.array(next_states)
                    self.q_values[state[X],state[Y],action] = np.sum(states_probability*(reward+self.discount*self.state_values[next_states[:,0], next_states[:,1]]))
    
                v = self.state_values[state[X],state[Y]]

                self.state_values[state[X],state[Y]] = np.max(self.q_values[state[X],state[Y],:])

                self.delta = np.max([self.delta, abs(v - self.state_values[state[X],state[Y]])])
    
    def get_optimal_policy(self, state):
        return np.argmax(self.q_values[state[X],state[Y],:])
    
    def get_state_values(self):
        return self.state_values
    
    def get_q_values(self):
        return self.q_values

In [4]:
class PolicyIteration():
    def __init__(self, env, discount, theta, map_size):
        self.env = env
        self.discount = discount
        self.theta = theta
        self.map_size = map_size
        self.delta = 0
        self.state_values = np.zeros((self.map_size,self.map_size))
        self.q_values = np.zeros((self.map_size,self.map_size,4))
        self.optimal_policy = np.zeros((self.map_size,self.map_size))
        self.policy_stable = False
    
    
    def policy_evaluation(self):
        self.delta = np.inf

        while(self.delta > self.theta):

            self.delta = 0

            for state in itertools.product(range(self.map_size), range(self.map_size)):
                if (state[X],state[Y]) == (self.map_size-1,self.map_size-1):
                    continue

                for action in ACTION_SPACE:
                    next_states, states_probability, breaking_possibility, is_end = self.env.find_next_states(action=action, current_state=(state[X],state[Y]))
                    reward = np.where(is_end, 99, ((1-breaking_possibility) * -1) + (breaking_possibility * -11))
                    next_states = np.array(next_states)
                    self.q_values[state[X],state[Y],action] = np.sum(states_probability*(reward+self.discount*self.state_values[next_states[:,0], next_states[:,1]]))

                v = self.state_values[state[X],state[Y]]

                self.state_values[state[X],state[Y]] = np.max(self.q_values[state[X],state[Y],:])

                self.delta = np.max([self.delta, abs(v - self.state_values[state[X],state[Y]])])
                
    def policy_improvement(self):
        self.policy_stable = True
        old_policy = self.optimal_policy.copy()
#         self.optimal_policy = self.get_state_values()
        for state in itertools.product(range(self.map_size), range(self.map_size)):
            self.optimal_policy[state[X],state[Y]] = self.get_optimal_policy([state[X],state[Y]])
        if not (old_policy == self.optimal_policy).all():
            self.policy_stable = False
    
    def policy_estimation(self):
        while(not self.policy_stable):
            self.policy_evaluation()
            self.policy_improvement()
    
    def get_optimal_policy(self, state):
        return np.argmax(self.q_values[state[X],state[Y],:])
    
    def get_state_values(self):
        return self.state_values
    
    def get_q_values(self):
        return self.q_values

<a name='2-1'></a>
## Part 1

In [3]:
from env import FrozenLake

MAP_SIZE = 6

environment = FrozenLake(studentNum=Sd_number)

environment.render()


-------------------------------------------------------
| 0.0000 | 1.0000 | 1.0000 | 1.0000 | 1.0000 | 1.0000 | 
-------------------------------------------------------
| 0.0001 | 0.0001 | 1.0000 | 1.0000 | 1.0000 | 1.0000 | 
-------------------------------------------------------
| 1.0000 | 0.0001 | 1.0000 | 1.0000 | 1.0000 | 1.0000 | 
-------------------------------------------------------
| 1.0000 | 0.0001 | 0.0001 | 0.0001 | 0.0001 | 0.0001 | 
-------------------------------------------------------
| 1.0000 | 1.0000 | 1.0000 | 1.0000 | 1.0000 | 0.0001 | 
-------------------------------------------------------
| 1.0000 | 1.0000 | 1.0000 | 1.0000 | 1.0000 | 0.0000 | 
-------------------------------------------------------


<a name='2-1-1'></a>
### Value Iteration

In [5]:
value_iteration = ValueIteration(FrozenLake(studentNum=Sd_number), Discount, 1e-8, MAP_SIZE)

start = time.time()
value_iteration.value_estimation()
end = time.time()

print("Convergence time:", end - start)

Convergence time: 0.6083495616912842


In [6]:
print("states value :\n", show_states_value_on_map(value_iteration.get_state_values(), MAP_SIZE))

states value :
 
------------------------------------------------------------
| 26.1929 | 30.0812 | 17.3231 | 22.7638 | 29.6611 | 36.8419 | 
------------------------------------------------------------
| 30.6258 | 35.6471 | 31.9859 | 38.4259 | 46.1634 | 54.3519 | 
------------------------------------------------------------
| 35.9200 | 41.8198 | 48.6298 | 56.2760 | 64.9640 | 74.3257 | 
------------------------------------------------------------
| 41.3085 | 48.5310 | 56.3703 | 65.1780 | 74.9210 | 85.6277 | 
------------------------------------------------------------
| 25.6742 | 41.4664 | 48.9862 | 64.8993 | 85.4633 | 97.6371 | 
------------------------------------------------------------
| 21.5512 | 37.0133 | 54.6250 | 74.5789 | 97.0272 | 0.0000 | 
------------------------------------------------------------


In [7]:
print("state-action value : \n\n", value_iteration.get_q_values())

state-action value : 

 [[[22.52333384 26.19285648 16.54290946 22.52333384]
  [22.25406176 30.08123284  5.70989766 16.27363738]
  [15.74603334 17.32309307  9.68720736  5.18229361]
  [ 5.29065143 22.76381015 15.50649154  9.79556517]
  [10.28617933 29.66106422 21.94287664 15.99710569]
  [16.39790527 36.841947   22.34367621 22.34367621]]

 [[26.46810978 21.65264356 30.62575822 22.79858714]
  [26.37839636 35.64707276 18.30550907 16.72844934]
  [30.43530202 31.98589435 23.53712556  6.06396684]
  [18.31372514 38.42594177 30.05272431 10.67783942]
  [24.18993242 46.16342959 37.37665554 16.9326138 ]
  [31.03361312 54.35193588 37.81369633 23.31542554]]

 [[21.83583513 26.29756297 35.91997776 26.65130135]
  [22.1788024  41.81981185 32.70250939 31.15191706]
  [36.58198018 48.62976881 39.35263313 19.2404165 ]
  [33.37498722 56.27601106 46.89971561 24.92621844]
  [40.32686655 64.96398184 55.27197185 31.95364909]
  [48.01700414 74.32565515 55.76846965 39.23023009]]

 [[26.12925635 13.18402328 41.3085

In [8]:
print("optimal policy :\n", show_actions_on_map(value_iteration, MAP_SIZE))

optimal policy :
 
-------------------------
| ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | → | → | → | → | ↓ | 
-------------------------
| → | ↑ | ↑ | → | → | ↓ | 
-------------------------
| → | → | → | → | → | * | 
-------------------------


<a name='2-1-2'></a>
### Policy Iteration

In [10]:
policy_iteration = PolicyIteration(FrozenLake(studentNum=Sd_number), Discount, 1e-8, MAP_SIZE)

start = time.time()
policy_iteration.policy_estimation()
end = time.time()

print("Convergence time:", end - start)

Convergence time: 0.5701236724853516


In [11]:
print("states value :\n", show_states_value_on_map(policy_iteration.get_state_values(), MAP_SIZE))

states value :
 
------------------------------------------------------------
| 26.1929 | 30.0812 | 17.3231 | 22.7638 | 29.6611 | 36.8419 | 
------------------------------------------------------------
| 30.6258 | 35.6471 | 31.9859 | 38.4259 | 46.1634 | 54.3519 | 
------------------------------------------------------------
| 35.9200 | 41.8198 | 48.6298 | 56.2760 | 64.9640 | 74.3257 | 
------------------------------------------------------------
| 41.3085 | 48.5310 | 56.3703 | 65.1780 | 74.9210 | 85.6277 | 
------------------------------------------------------------
| 25.6742 | 41.4664 | 48.9862 | 64.8993 | 85.4633 | 97.6371 | 
------------------------------------------------------------
| 21.5512 | 37.0133 | 54.6250 | 74.5789 | 97.0272 | 0.0000 | 
------------------------------------------------------------


In [12]:
print("state-action value : \n\n", policy_iteration.get_q_values())

state-action value : 

 [[[22.52333384 26.19285648 16.54290946 22.52333384]
  [22.25406176 30.08123284  5.70989766 16.27363738]
  [15.74603334 17.32309307  9.68720736  5.18229361]
  [ 5.29065143 22.76381015 15.50649154  9.79556517]
  [10.28617933 29.66106422 21.94287664 15.99710569]
  [16.39790527 36.841947   22.34367621 22.34367621]]

 [[26.46810978 21.65264356 30.62575822 22.79858714]
  [26.37839636 35.64707276 18.30550907 16.72844934]
  [30.43530202 31.98589435 23.53712556  6.06396684]
  [18.31372514 38.42594177 30.05272431 10.67783942]
  [24.18993242 46.16342959 37.37665554 16.9326138 ]
  [31.03361312 54.35193588 37.81369633 23.31542554]]

 [[21.83583513 26.29756297 35.91997776 26.65130135]
  [22.1788024  41.81981185 32.70250939 31.15191706]
  [36.58198018 48.62976881 39.35263313 19.2404165 ]
  [33.37498722 56.27601106 46.89971561 24.92621844]
  [40.32686655 64.96398184 55.27197185 31.95364909]
  [48.01700414 74.32565515 55.76846965 39.23023009]]

 [[26.12925635 13.18402328 41.3085

In [13]:
print("optimal policy :\n", show_actions_on_map(policy_iteration, MAP_SIZE))

optimal policy :
 
-------------------------
| ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | → | → | → | → | ↓ | 
-------------------------
| → | ↑ | ↑ | → | → | ↓ | 
-------------------------
| → | → | → | → | → | * | 
-------------------------


<a name='2-2'></a>
## Part 2

In [2]:
from env import FrozenLake

MAP_SIZE = 6

environment = FrozenLake(studentNum=Sd_number)

environment.render()


-------------------------------------------------------
| 0.0000 | 0.8444 | 0.3343 | 0.3058 | 0.8208 | 0.4955 | 
-------------------------------------------------------
| 0.0010 | 0.0010 | 0.7536 | 0.4510 | 0.8006 | 0.3135 | 
-------------------------------------------------------
| 0.3768 | 0.0010 | 0.8268 | 0.8268 | 0.8908 | 0.5312 | 
-------------------------------------------------------
| 0.6320 | 0.0010 | 0.0010 | 0.0010 | 0.0010 | 0.0010 | 
-------------------------------------------------------
| 0.9490 | 0.8463 | 0.6388 | 0.1015 | 0.2757 | 0.0010 | 
-------------------------------------------------------
| 0.6810 | 0.1309 | 0.2624 | 0.1043 | 0.1053 | 0.0000 | 
-------------------------------------------------------


<a name='2-2-1'></a>
### Value Iteration

In [7]:
value_iteration = ValueIteration(FrozenLake(studentNum=Sd_number), Discount, 1e-8, MAP_SIZE)

start = time.time()
value_iteration.value_estimation()
end = time.time()

print("Convergence time:", end - start)

Convergence time: 2.063612461090088


In [8]:
print("states value :\n", show_states_value_on_map(value_iteration.get_state_values(), MAP_SIZE))

states value :
 
------------------------------------------------------------
| 4.1375 | 5.4381 | 0.0744 | 5.8946 | 11.2824 | 22.1840 | 
------------------------------------------------------------
| 7.5472 | 10.7228 | 10.4150 | 15.8092 | 23.9532 | 35.2093 | 
------------------------------------------------------------
| 12.5000 | 17.8140 | 24.7531 | 33.0679 | 43.1218 | 54.3110 | 
------------------------------------------------------------
| 16.5526 | 24.9331 | 34.6425 | 45.5745 | 57.5673 | 71.6122 | 
------------------------------------------------------------
| 13.2761 | 27.7960 | 43.6927 | 57.2155 | 73.8153 | 89.9020 | 
------------------------------------------------------------
| 25.1811 | 37.5886 | 53.8614 | 70.5729 | 89.5710 | 0.0000 | 
------------------------------------------------------------


In [9]:
print("state-action value : \n\n", value_iteration.get_q_values())

state-action value : 

 [[[ 2.30220940e+00  4.13745319e+00 -2.06202519e+00  2.30220940e+00]
  [ 1.88803968e+00  5.43814149e+00 -2.31183670e+00 -2.47619491e+00]
  [-3.40413257e+00 -1.71691181e-01  7.43710490e-02 -3.23977437e+00]
  [-1.90202090e+00  5.89463646e+00  1.23156755e+00  1.41212452e+00]
  [ 3.62422918e+00  1.04070850e+01  1.12823679e+01  3.44367221e+00]
  [ 6.22010270e+00  2.21839597e+01  1.40587983e+01  1.40587983e+01]]

 [[ 5.83230588e+00  6.25184185e+00  7.54716391e+00  3.99706209e+00]
  [ 5.17871381e+00  1.07228269e+01  2.21167682e+00 -1.02076457e+00]
  [ 7.79357191e+00  1.04150371e+01  7.84025107e+00  4.35937201e-02]
  [ 4.01550194e+00  1.58092180e+01  1.10444200e+01  4.26156417e+00]
  [ 1.18429922e+01  2.39531690e+01  2.31437803e+01  7.17992325e+00]
  [ 1.71997876e+01  3.52092904e+01  2.62002319e+01  1.80750705e+01]]

 [[ 7.37545264e+00  8.03308254e+00  1.25000298e+01  6.95591667e+00]
  [ 8.84517058e+00  1.78140400e+01  1.27619578e+01  1.01404926e+01]
  [ 1.56657639e+01  

In [10]:
print("optimal policy :\n", show_actions_on_map(value_iteration, MAP_SIZE))

optimal policy :
 
-------------------------
| ↓ | ↓ | → | ↓ | → | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | → | → | → | → | ↓ | 
-------------------------
| → | → | → | → | → | ↓ | 
-------------------------
| → | → | → | → | → | * | 
-------------------------


<a name='2-2-2'></a>
### Policy Iteration

In [11]:
from env import FrozenLake

policy_iteration = PolicyIteration(FrozenLake(studentNum=Sd_number), Discount, 1e-8, MAP_SIZE)

start = time.time()
policy_iteration.policy_estimation()
end = time.time()

print("Convergence time:", end - start)

Convergence time: 1.35288405418396


In [12]:
print("states value :\n", show_states_value_on_map(policy_iteration.get_state_values(), MAP_SIZE))

states value :
 
------------------------------------------------------------
| 4.1375 | 5.4381 | 0.0744 | 5.8946 | 11.2824 | 22.1840 | 
------------------------------------------------------------
| 7.5472 | 10.7228 | 10.4150 | 15.8092 | 23.9532 | 35.2093 | 
------------------------------------------------------------
| 12.5000 | 17.8140 | 24.7531 | 33.0679 | 43.1218 | 54.3110 | 
------------------------------------------------------------
| 16.5526 | 24.9331 | 34.6425 | 45.5745 | 57.5673 | 71.6122 | 
------------------------------------------------------------
| 13.2761 | 27.7960 | 43.6927 | 57.2155 | 73.8153 | 89.9020 | 
------------------------------------------------------------
| 25.1811 | 37.5886 | 53.8614 | 70.5729 | 89.5710 | 0.0000 | 
------------------------------------------------------------


In [13]:
print("state-action value : \n\n", policy_iteration.get_q_values())

state-action value : 

 [[[ 2.30220940e+00  4.13745319e+00 -2.06202519e+00  2.30220940e+00]
  [ 1.88803968e+00  5.43814149e+00 -2.31183670e+00 -2.47619491e+00]
  [-3.40413257e+00 -1.71691181e-01  7.43710490e-02 -3.23977437e+00]
  [-1.90202090e+00  5.89463646e+00  1.23156755e+00  1.41212452e+00]
  [ 3.62422918e+00  1.04070850e+01  1.12823679e+01  3.44367221e+00]
  [ 6.22010270e+00  2.21839597e+01  1.40587983e+01  1.40587983e+01]]

 [[ 5.83230588e+00  6.25184185e+00  7.54716391e+00  3.99706209e+00]
  [ 5.17871381e+00  1.07228269e+01  2.21167682e+00 -1.02076457e+00]
  [ 7.79357191e+00  1.04150371e+01  7.84025107e+00  4.35937201e-02]
  [ 4.01550194e+00  1.58092180e+01  1.10444200e+01  4.26156417e+00]
  [ 1.18429922e+01  2.39531690e+01  2.31437803e+01  7.17992325e+00]
  [ 1.71997876e+01  3.52092904e+01  2.62002319e+01  1.80750705e+01]]

 [[ 7.37545264e+00  8.03308254e+00  1.25000298e+01  6.95591667e+00]
  [ 8.84517058e+00  1.78140400e+01  1.27619578e+01  1.01404926e+01]
  [ 1.56657639e+01  

In [14]:
print("optimal policy :\n", show_actions_on_map(policy_iteration, MAP_SIZE))

optimal policy :
 
-------------------------
| ↓ | ↓ | → | ↓ | → | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | → | → | → | → | ↓ | 
-------------------------
| → | → | → | → | → | ↓ | 
-------------------------
| → | → | → | → | → | * | 
-------------------------


<a name='2-3'></a>
## Part 3

In [37]:
from env import FrozenLake

MAP_SIZE = 15

environment = FrozenLake(studentNum=Sd_number)

environment.render()


-------------------------------------------------------------------------------------------------------------------------------------------
| 0.0000 | 0.4520 | 0.5950 | 0.6061 | 0.7932 | 0.7197 | 0.2585 | 0.6634 | 0.5232 | 0.0378 | 0.7218 | 0.9371 | 0.6105 | 0.7366 | 0.6932 | 
-------------------------------------------------------------------------------------------------------------------------------------------
| 0.0010 | 0.0010 | 0.9939 | 0.9846 | 0.0024 | 0.9573 | 0.6653 | 0.9827 | 0.4317 | 0.7456 | 0.0247 | 0.3842 | 0.2294 | 0.6634 | 0.4343 | 
-------------------------------------------------------------------------------------------------------------------------------------------
| 0.8383 | 0.0010 | 0.0010 | 0.0010 | 0.0010 | 0.4788 | 0.3056 | 0.9012 | 0.6050 | 0.6653 | 0.6583 | 0.2466 | 0.9152 | 0.7013 | 0.4871 | 
-------------------------------------------------------------------------------------------------------------------------------------------
| 0.1692 | 0.5775 | 0.107

<a name='2-3-1'></a>
### A

In [20]:
value_iteration = ValueIteration(FrozenLake(studentNum=Sd_number), Discount, 1e-8, MAP_SIZE)

start = time.time()
value_iteration.value_estimation()
end = time.time()

print("Convergence time:", end - start)

Convergence time: 50.2479248046875


In [21]:
print("states value :\n", show_states_value_on_map(value_iteration.get_state_values(), MAP_SIZE))

states value :
 
------------------------------------------------------------------------------------------------------------------------------------------------------
| -16.3540 | -17.7709 | -24.1914 | -29.6072 | -24.8326 | -32.5849 | -36.9063 | -37.5100 | -33.1643 | -32.6295 | -32.5476 | -34.9645 | -36.2839 | -37.5653 | -33.0312 | 
------------------------------------------------------------------------------------------------------------------------------------------------------
| -16.9637 | -18.2584 | -19.9409 | -21.3113 | -22.0070 | -24.4265 | -31.7104 | -37.2473 | -37.3787 | -32.2620 | -32.9630 | -31.2130 | -34.2867 | -32.1619 | -27.8441 | 
------------------------------------------------------------------------------------------------------------------------------------------------------
| -18.3380 | -19.1476 | -19.5022 | -19.4632 | -18.9106 | -21.0530 | -27.7783 | -31.9602 | -35.9328 | -33.2016 | -26.7636 | -28.7244 | -28.7113 | -27.1813 | -21.0643 | 
--------------------------

In [22]:
print("state-action value : \n\n", value_iteration.get_q_values())

state-action value : 

 [[[-1.63540219e+01 -1.66892577e+01 -1.98311992e+01 -1.63540219e+01]
  [-1.77709255e+01 -1.88053064e+01 -2.55732025e+01 -2.12481029e+01]
  [-2.41913837e+01 -2.86146919e+01 -3.15077868e+01 -2.85164833e+01]
  [-2.96071985e+01 -3.03894085e+01 -3.11423887e+01 -3.25985020e+01]
  [-3.25590710e+01 -2.48325964e+01 -3.48485620e+01 -3.11029577e+01]
  [-3.25848939e+01 -3.33502334e+01 -3.58969237e+01 -3.63304983e+01]
  [-3.77052282e+01 -3.69063407e+01 -4.00266265e+01 -3.72716537e+01]
  [-3.79429318e+01 -4.24718354e+01 -3.75099582e+01 -4.06979046e+01]
  [-3.95531621e+01 -3.80924985e+01 -3.31643287e+01 -3.63652157e+01]
  [-3.58303568e+01 -3.66778225e+01 -3.66889671e+01 -3.26294698e+01]
  [-3.25476101e+01 -3.26491575e+01 -3.92041443e+01 -3.66071075e+01]
  [-3.77106360e+01 -3.49644619e+01 -3.90605342e+01 -4.03076729e+01]
  [-4.08959941e+01 -3.62838742e+01 -4.10977936e+01 -3.96488555e+01]
  [-3.94737593e+01 -3.75652767e+01 -3.82138221e+01 -4.09226974e+01]
  [-4.00949815e+01 -3.30

In [23]:
print("optimal policy :\n", show_actions_on_map(value_iteration, MAP_SIZE))

optimal policy :
 
----------------------------------------------------------------
| ← | ← | ← | ← | ↓ | ← | ↓ | → | → | ↑ | ← | ↓ | ↓ | ↓ | ↓ | 
----------------------------------------------------------------
| ↑ | ← | ← | ↓ | ↓ | ← | ↓ | ← | ↑ | ↑ | ↓ | ↓ | ← | → | ↓ | 
----------------------------------------------------------------
| ↑ | ↑ | ← | → | ↓ | ← | ← | ← | ↓ | → | ↓ | ↓ | ↓ | → | ↓ | 
----------------------------------------------------------------
| → | ↑ | ↑ | → | ↑ | ← | ← | ↓ | ← | → | ↓ | ↓ | ↓ | → | ↓ | 
----------------------------------------------------------------
| ↑ | → | ↑ | → | ↑ | ↓ | ↓ | ↓ | ↓ | ↓ | → | ↓ | ← | ↓ | ↓ | 
----------------------------------------------------------------
| ↓ | ↓ | ↑ | → | → | ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | 
----------------------------------------------------------------
| ← | ← | ← | ↑ | → | → | → | → | → | → | → | ↓ | ← | ↓ | ↓ | 
----------------------------------------------------------------
| ↑ | ← | → | → | → 

<a name='2-3-2'></a>
### B

In [38]:
value_iteration = ValueIteration(FrozenLake(studentNum=Sd_number), Discount, 1e-8, MAP_SIZE)

start = time.time()
value_iteration.value_estimation()
end = time.time()

print("Convergence time:", end - start)

Convergence time: 32.790149211883545


In [39]:
print("states value :\n", show_states_value_on_map(value_iteration.get_state_values(), MAP_SIZE))

states value :
 
------------------------------------------------------------------------------------------------------------------------------------------------------
| -9.4494 | -9.6828 | -15.5957 | -14.2689 | -5.8359 | -14.5395 | -12.2541 | -4.8056 | 5.2702 | 11.6100 | 24.3183 | 31.4742 | 37.8398 | 45.5492 | 58.3306 | 
------------------------------------------------------------------------------------------------------------------------------------------------------
| -8.6000 | -7.1480 | -4.9839 | -1.8561 | 0.6635 | -3.3817 | -3.3516 | -0.2640 | 8.8562 | 23.6576 | 33.5267 | 46.0412 | 51.4161 | 64.2515 | 79.1978 | 
------------------------------------------------------------------------------------------------------------------------------------------------------
| -6.9564 | -4.1271 | -0.2894 | 3.8515 | 8.5749 | 6.2034 | 5.4243 | 12.6980 | 20.3503 | 33.3540 | 51.6270 | 62.6684 | 74.8393 | 88.4228 | 106.7267 | 
-------------------------------------------------------------------------

In [40]:
print("state-action value : \n\n", value_iteration.get_q_values())

state-action value : 

 [[[-9.90199910e+00 -9.44935994e+00 -1.27400892e+01 -9.90199910e+00]
  [-1.09195007e+01 -9.68276861e+00 -1.78086032e+01 -1.37575908e+01]
  [-1.55956580e+01 -1.63098281e+01 -1.89969795e+01 -1.96466705e+01]
  [-1.93507777e+01 -1.42688904e+01 -1.52694535e+01 -1.87010867e+01]
  [-1.75218012e+01 -5.83591808e+00 -1.83494661e+01 -1.40901680e+01]
  [-1.48801619e+01 -1.45395003e+01 -1.51381704e+01 -1.91394600e+01]
  [-1.86222380e+01 -1.22540672e+01 -1.30278186e+01 -1.46209483e+01]
  [-1.26810077e+01 -1.05511375e+01 -4.80564297e+00 -1.10878780e+01]
  [-7.34764408e+00  1.41957561e+00  5.27015072e+00 -1.06540902e+00]
  [ 2.51560964e+00  1.11101218e+01  1.16099590e+01  8.85116938e+00]
  [ 1.24047108e+01  2.43182761e+01  1.77357864e+01  1.51635004e+01]
  [ 1.77183669e+01  3.14742444e+01  2.56876744e+01  2.02906528e+01]
  [ 2.28250769e+01  3.78398003e+01  3.16282033e+01  2.82220984e+01]
  [ 3.16042386e+01  4.55492183e+01  4.21727806e+01  3.50103435e+01]
  [ 3.83460510e+01  5.83

In [41]:
print("optimal policy :\n", show_actions_on_map(value_iteration, MAP_SIZE))

optimal policy :
 
----------------------------------------------------------------
| ↓ | ↓ | ← | ↓ | ↓ | ↓ | ↓ | → | → | → | ↓ | ↓ | ↓ | ↓ | ↓ | 
----------------------------------------------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | → | → | → | ↓ | ↓ | ↓ | ↓ | ↓ | 
----------------------------------------------------------------
| → | → | → | → | ↓ | ↓ | ↓ | ↓ | ↓ | → | ↓ | ↓ | ↓ | ↓ | ↓ | 
----------------------------------------------------------------
| → | → | → | → | ↓ | ↓ | ↓ | ↓ | ↓ | → | ↓ | ↓ | ↓ | ↓ | ↓ | 
----------------------------------------------------------------
| ↓ | → | → | → | ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | → | ↓ | ↓ | ↓ | ↓ | 
----------------------------------------------------------------
| ↓ | → | → | → | → | ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | 
----------------------------------------------------------------
| → | → | → | → | → | → | → | → | → | → | → | ↓ | ↓ | ↓ | ↓ | 
----------------------------------------------------------------
| ↓ | ↓ | → | → | → 